In [ ]:
import copy
import spacy
nlp_en = spacy.load('en')
nlp_de = spacy.load('de')
from spacy.en import English
parser = English()

In [ ]:
from spacy.en import English
parser = English()

In [ ]:
import xml.etree.ElementTree as ET

In [ ]:
def create_prompt_response_map(gram):
    prompt_response_map = {}
    for prompt_unit in gram:
        key = None
        responses = []
        for prompt in prompt_unit:
            if prompt.tag == None:
                print("None")
            else:
                if(prompt.tag == 'prompt'):
                    key = prompt.text
                elif(prompt.tag == 'response'):
                    responses.append(prompt.text)
        prompt_response_map[key] = responses
    return prompt_response_map

def read_grammar_and_create_map(file):
    tree = ET.parse(file)
    grammar = tree.getroot()
    return create_prompt_response_map(grammar)

def get_sag_prompts(prompt_response_map):
    sag_prompts = {}
    for key in prompt_response_map:
        if "Sag" in key:
            sag_prompts[key] = prompt_response_map[key]
    return sag_prompts

    
grammar = read_grammar_and_create_map('referenceGrammar.xml')
#sag_prompts = get_sag_prompts(prompt_response_map)

In [ ]:
test_file_csv = "textProcessing_testKaldi.csv"
test_data = None
with open(test_file_csv, 'r') as reader:
    test_data = reader.readlines()  
    
def prompt_map_test_data():
    prompt_map = {}
    for item in test_data[1:]:
        split = item.replace("\n", "").split("\t")
        prompt = split[1]
        
        if prompt in prompt_map:
            prompt_map[prompt].append({'id': split[0], "transcript": split[3], "prompt": split[1]})
        else:
            arr = []
            arr.append({'id':split[0], "transcript":split[3], "prompt": split[1]})
            prompt_map[prompt] = arr
            
    return prompt_map
prompt_test_map = prompt_map_test_data()

# Create Basis based on Reference Grammar and Normal Case

In [ ]:
special_case_map = {"don't": "do not",
 "haven't" : "have not",
 "i'd" : "i would",
 "i'm" : "i am",
 "isn't": "is not",
 "it's": "it is",
 "o'clock": "o'clock",
 "that's" :"that is",
 "there's": "there is"}

def sentence_to_normal_case(sentence):
    trans_sentence = ""
    words = sentence.split(" ")
    for word in words:
        if word in special_case_map:
            word = special_case_map[word]
        trans_sentence += word + " "
    
    return trans_sentence[:-1]

meaning_false = {}
for prompt_unit in prompt_test_map:
    for response in prompt_test_map[prompt_unit]:
        
        sentence = response['transcript']
        if "***" in sentence:
            continue
        if "'" in sentence:
            sentence = sentence_to_normal_case(sentence)
            #print(dict_prompt['transcript'])
            
        sentence = ' '.join(sentence.split())
        if sentence not in prompt_response_map[prompt_unit]:
            meaning_false[response['id']] = {"sentence":sentence, "prompt":response['prompt']}

In [ ]:
prompt_meaning_false = {}
for id_ in meaning_false:
    for item in meaning_false[id_]:
        prompt_key = meaning_false[id_]["prompt"]
        if prompt_key in prompt_meaning_false:
            prompt_meaning_false[prompt_key].append(meaning_false[id_]["sentence"])
        else:
            arr = []
            arr.append(meaning_false[id_]["sentence"])
            prompt_meaning_false[prompt_key] = arr


# Extract Meaning Words

In [ ]:
def nlp_sentence(sentence):
    parsed = parser(sentence)
    lemmas = []
    words = []
    tags = []
    poss = []
    for i, token in enumerate(parsed):
        lemma = token.lemma_
        words.append(token)
        lemmas.append(lemma)
        tag = token.tag_
        pos = token.tag_
        tags.append(tag)
        poss.append(pos)
    return words, lemmas, tags, poss

In [ ]:
len(meaning_false)

In [ ]:
test_key = "Sag: Ich habe keine Reservation"
prompt_meaning_false[test_key]

In [ ]:
def replace_sentence_emp_words(sentence):
    return sentence.replace("thank you ", "")\
    .replace(" please", "")\
    .replace("yes ", "")\
    .replace("thank you ", "")\
    .replace("no ", "")

In [ ]:
def spacy_words_to_string_array(words):
    arr = []
    for word in words:
        arr.append(str(word))
    return arr

In [ ]:
def extract_key_nouns(nlp_sent):
    tags = nlp_sent[2]
    words = spacy_words_to_string_array(nlp_sent[0])
    sentence = " ".join(words) + " "
    #words = " ".join(nlp_sent[0])

    tags_string = " ".join(tags)
    patterns = [['CD', 'JJ', 'NN'], ['JJ', 'NN'], ['DT', 'NN'], ['CD', 'JJ', 'NNS'], ['JJ', 'NNS'], ['DT', 'NNS']]
    extracted_words = []
    for pattern in patterns:
        pattern_string = " ".join(pattern) + " "
        if pattern_string in tags_string: 
            if pattern[-1] not in tags and pattern[0] not in tags:
                continue
            pattern_start_index = tags.index(pattern[0])
            pattern_end_index = tags.index(pattern[-1])
            if pattern_end_index - pattern_start_index + 1 == len(pattern):
                extracted_part = ""
                for i in range(pattern_start_index, pattern_end_index+1):
                    extracted_part += words[i] + " "
                
                extracted_words.append(extracted_part[:-1])
    return extracted_words

#print(nlp_sentence("i am looking for one musical ticket")[2])    
extract_key_nouns(nlp_sentence("i am looking for one musical ticket"))

In [ ]:
counter = 0
result = []
for sentence in grammar[test_key]:
    replaced_sentence = replace_sentence_emp_words(sentence)
    nlp_sent = nlp_sentence(replaced_sentence)
    tags = nlp_sent[2]
    if tags[0] == 'PRP':
        counter +=1
        result.extend(extract_key_nouns(nlp_sent))
        print(str(spacy_words_to_string_array(nlp_sent[0])))
        print(nlp_sent[2])
print(counter)
print(len(result))
print(set(result))

In [ ]:
def extract_key_verbs_by_prp(nlp_sent):
    tags = nlp_sent[2]
    words = spacy_words_to_string_array(nlp_sent[0])
    sentence = " ".join(words)
    #words = " ".join(nlp_sent[0])

    tags_string = " ".join(tags)
    prio_one_patterns = [['VBP', 'VBG', 'IN'], ['MD', 'VB', 'CD'], ['VBP', 'RB']]
    # Only when no prio_one_pattern fits
    prio_sec_patterns = [['VBP']]
    match = False
    extracted_words = []
    for pattern in prio_one_patterns:
        pattern_string = " ".join(pattern)
        if pattern_string in tags_string:  
            pattern_start_index = tags.index(pattern[0])
            pattern_end_index = tags.index(pattern[-1])
            if pattern_end_index - pattern_start_index + 1 == len(pattern):
                extracted_part = ""
                for i in range(pattern_start_index, pattern_end_index+1):
                    extracted_part += words[i] + " "
                
                extracted_words.append(extracted_part[:-1])
    if len(extracted_words) <1:
        for pattern in prio_sec_patterns:
            pattern_string = " ".join(pattern)
            if pattern_string in tags_string:  
                pattern_start_index = tags.index(pattern[0])
                pattern_end_index = tags.index(pattern[-1])
                if pattern_end_index - pattern_start_index + 1 == len(pattern):
                    extracted_part = ""
                    for i in range(pattern_start_index, pattern_end_index+1):
                        extracted_part += words[i] + " "

                    extracted_words.append(extracted_part[:-1])
        
    return extracted_words

extract_key_verbs_by_prp(nlp_sentence("i am looking for one musical ticket"))

In [ ]:
def extract_key_questions(nlp_sent):
    tags = nlp_sent[2]
    words = spacy_words_to_string_array(nlp_sent[0])
    sentence = " ".join(words)
    #words = " ".join(nlp_sent[0])

    tags_string = " ".join(tags)
    prio_one_patterns = [['MD', 'PRP', 'VB'], ['VBP', 'PRP', 'VBN'], ['VBP', 'PRP', 'VB']]
    # Only when no prio_one_pattern fits
    #prio_sec_patterns = [['VBP']]
    match = False
    extracted_words = []
    for pattern in prio_one_patterns:
        pattern_string = " ".join(pattern) + " "
        if pattern_string in tags_string:
            if pattern[0] not in tags and pattern[-1] not in tags:
                continue
            pattern_start_index = tags.index(pattern[0])
            pattern_end_index = tags.index(pattern[-1])
            if pattern_end_index - pattern_start_index + 1 == len(pattern):
                extracted_part = ""
                for i in range(pattern_start_index, pattern_end_index+1):
                    extracted_part += words[i] + " "
                
                extracted_words.append(extracted_part[:-1])
        
    return extracted_words

_test_key = "have you got one musical ticket"
#print(nlp_sentence(_test_key)[2])
extract_key_questions(nlp_sentence(_test_key))


In [ ]:
counter = 0
result_prp = []
result_md = []
for sentence in grammar[test_key]:
    replaced_sentence = replace_sentence_emp_words(sentence)
    nlp_sent = nlp_sentence(replaced_sentence)
    tags = nlp_sent[2]
    if tags[0] != 'PRP':
        extracted = extract_key_questions(nlp_sent)
        if len(extracted) < 1:
            print(sentence)
            print(nlp_sent[2])
            #print(extract_key_verbs_by_prp(nlp_sent))
        else:
            result_md.extend(extracted)
            

        #print(str(spacy_words_to_string_array(nlp_sent[0])))
        #print(nlp_sent[2])
print(counter)
print(len(result_prp))
print(set(result_md))

# Combine NN, Q, V

In [ ]:
extract_key_nouns(nlp_sentence("i do not have a reservation"))

In [ ]:
for sent in errors_nouns:
    n_s = nlp_sentence(sent)
    try:    
        extract_key_nouns(n_s)
    except:
        print(sent)
        print(n_s)
    

In [ ]:
verbs = []
nouns = []
errors_nouns = []
errors_verbs = []
for key in grammar:
    for response in grammar[key]:
        replaced_sentence = replace_sentence_emp_words(response)
        nlp_sent = nlp_sentence(replaced_sentence)  
        tags = nlp_sent[2]
        if tags[0] == 'PRP':
            try:
                verbs.extend(extract_key_verbs_by_prp(nlp_sent))
            except:
                errors_verbs.append(response)
            try:    
                nouns.extend(extract_key_nouns(nlp_sent))
            except:
                errors_nouns.append(response)
            
        
print(len(set(verbs)))
print(len(set(nouns)))

In [ ]:
for verb in set(verbs):
    print(verb)

In [ ]:
verbs = []
nouns = []
for response in grammar[test_key]:
    replaced_sentence = replace_sentence_emp_words(response)
    nlp_sent = nlp_sentence(replaced_sentence)  
    tags = nlp_sent[2]
    if tags[0] == 'PRP':
        verbs.extend(extract_key_verbs_by_prp(nlp_sent))
        nouns.extend(extract_key_nouns(nlp_sent))
        
        
print(set(verbs))
print(set(nouns))

In [ ]:
for item in prompt_test_map[test_key]:
    if item['transcript'] not in grammar[test_key]:
        print(item['transcript'])

In [ ]:
sag_prompts = get_sag_prompts(grammar)
for key in sag_prompts:
    print(key)

# Cluster approach

In [ ]:
test_prompts = ["Sag: Ich möchte mit Dollars bezahlen",
"Sag: Ich möchte mit Euros bezahlen",
"Sag: Ich möchte mit Kreditkarte bezahlen",
"Sag: Ich möchte mit Mastercard bezahlen",
"Sag: Ich möchte mit Postkarte bezahlen",
"Sag: Ich möchte mit Visa bezahlen"]

In [ ]:
test_prompts_sentences = []
counter_max = 0
max_prompt = ""
for test_prompt in test_prompts:
    tp_prompts = grammar[test_prompt]
    if len(tp_prompts) > counter_max:
        max_prompt = test_prompt
        counter_max = len(tp_prompts)
        
    test_prompts_sentences.extend(tp_prompts)
print(len(test_prompts_sentences))
print(counter_max)
print(max_prompt)

In [ ]:
def is_slice_in_list(s,l):
    len_s = len(s) #so we don't recompute length of s on every iteration
    return any(s == l[i:len_s+i] for i in range(len(l) - len_s+1))

def extract_by_pattern(patterns, tags, words):
    extracted_words = []
    tags_string = " ".join(tags)
    for pattern in patterns:
        pattern_string = " ".join(pattern)
        if is_slice_in_list(pattern, tags):
            pattern_start_index = tags.index(pattern[0])
            if len(pattern) > 1:
                pattern_end_index = tags.index(pattern[-1], pattern_start_index+1)
                if pattern_end_index - pattern_start_index + 1 == len(pattern):
                    extracted_part = ""
                    for i in range(pattern_start_index, pattern_end_index+1):
                        extracted_part += words[i] + " "

                    extracted_words.append(extracted_part[:-1])
            else:
                extracted_words.append(words[pattern_start_index])
    return extracted_words
    
def extract_key_nouns_credit_card(nlp_sent):
    tags = nlp_sent[2]
    words = spacy_words_to_string_array(nlp_sent[0])
    sentence = " ".join(words)
    #words = " ".join(nlp_sent[0])

    prio_one_patterns = [['NN', 'NNS'], ['NN', 'NN']]
    
    # Only when no prio_one_pattern fits
    prio_sec_patterns = [['NN'], ['NNS']]
    match = False
    extracted_words = extract_by_pattern(prio_one_patterns, tags, words)
    if len(extracted_words) < 1:
        extracted_words = extract_by_pattern(prio_sec_patterns, tags, words)
    
    return extracted_words

def extract_key_verbs_question_credit_card(nlp_sent):
    tags = nlp_sent[2]
    words = spacy_words_to_string_array(nlp_sent[0])
    sentence = " ".join(words)
    #words = " ".join(nlp_sent[0])
    
    prio_one_question_patterns = [['VBZ', 'PRP', 'JJ', 'TO', 'VB']]
    prio_two_question_patterns = [['MD', 'PRP', 'VB', 'IN']]
    prio_three_question_patterns = [['MD', 'PRP', 'VB'], ['VBP', 'PRP', 'VB']]
    
    #prio_one_patterns = [['VBP', 'VBG', 'IN'], ['MD', 'VB', 'CD'], ['VBP', 'RB']]
    # Only when no prio_one_pattern fits
    #prio_sec_patterns = [['VBP']]

    #prio_one_patterns = [['NN', 'NNS'], ['NN', 'NN']]
    
    # Only when no prio_one_pattern fits
    #prio_sec_patterns = [['NN'], ['NNS']]
    match = False
    extracted_words = extract_by_pattern(prio_one_question_patterns, tags, words)
    if len(extracted_words) < 1:
        extracted_words = extract_by_pattern(prio_two_question_patterns, tags, words)
        if len(extracted_words) < 1:
            extracted_words = extract_by_pattern(prio_three_question_patterns, tags, words)
    
    return extracted_words

def extract_key_verbs_credit_card(nlp_sent):
    tags = nlp_sent[2]
    words = spacy_words_to_string_array(nlp_sent[0])
    sentence = " ".join(words)
    #words = " ".join(nlp_sent[0])
    
    prio_one = [['MD', 'VB', 'TO', 'VB', 'IN']]

    
    #prio_one_patterns = [['VBP', 'VBG', 'IN'], ['MD', 'VB', 'CD'], ['VBP', 'RB']]
    # Only when no prio_one_pattern fits
    #prio_sec_patterns = [['VBP']]

    #prio_one_patterns = [['NN', 'NNS'], ['NN', 'NN']]
    
    # Only when no prio_one_pattern fits
    #prio_sec_patterns = [['NN'], ['NNS']]
    match = False
    extracted_words = extract_by_pattern(prio_one, tags, words)
    """if len(extracted_words) < 1:
        extracted_words = extract_by_pattern(prio_two_question_patterns, tags, words)
        if len(extracted_words) < 1:
            extracted_words = extract_by_pattern(prio_three_question_patterns, tags, words)
    """
    return extracted_words
    


In [472]:
testtt = "can i pay by card"
print(nlp_sentence(testtt))
extract_key_verbs_question_credit_card(nlp_sentence(testtt))

([can, i, pay, by, card], ['can', 'i', 'pay', 'by', 'card'], ['MD', 'PRP', 'VB', 'IN', 'NN'], ['MD', 'PRP', 'VB', 'IN', 'NN'])


['can i pay by']

In [ ]:
testtt = "can i pay by credit card"
print(nlp_sentence(testtt))
extract_key_nouns_credit_card(nlp_sentence(testtt))

In [ ]:
test_key = "Sag: Ich möchte mit Kreditkarte bezahlen"
verbs = []
pattern_verbs = []
for item in grammar[test_key]:
    nlp_se = nlp_sentence(item)
    verb_ques = extract_key_verbs_question_credit_card(nlp_se)
    if len(verb_ques) > 0:
        verbs.extend(verb_ques)
    else:
        print(item)
        verb = extract_key_verbs_credit_card(nlp_se)
        if len(verb) == 0:
            pattern_verbs.append(nlp_se[2])
        else:
            pattern_verbs.append(verb)

print(set(verbs))
print(pattern_verbs)

In [ ]:
nouns = []
question_verbs = []
verbs = []
for key in test_prompts:
    print(key)
    for item in grammar[key]:
        nlp_se = nlp_sentence(item)
        
        nouns.extend(extract_key_nouns_credit_card(nlp_se))
        question_verbs.extend(extract_key_verbs_question_credit_card(nlp_se))
        verbs.extend(extract_key_verbs_credit_card(nlp_se))
print(set(nouns))
print(set(question_verbs))
print(set(verbs))

In [ ]:
nouns_list = list(set(verbs))
print(nouns_list)

['can i buy by post ',
 'i would to pay by dollars',
 'i would pay with euros',
 'can i   am a    room',
 'my',
 'i want to pay with credit card',
 'i want to pay with by credit card',
 'i want to pay with  ***  credit card',
 ' i would like pay with by credit card',
 'i would to pay by credit card',
 'i would to pay by credit card',
 'i would to pay pay by credit card',
 'i would pay by credit card',
 'i   ***  would pay with credit card',
 'i want where is the credit card',
 'i would pay with credit card',
 'i would like pay with the credit card',
 'i want pay with credit card',
 'i want pay with credit card',
 'i like to buy with credit card',
 'i like to pay by credit card',
 'i like to pay with credit card',
 'i like to pay with credit card',
 'can i  ***  will credit card',
 " i don't note",
 'can i   am a pay by mastercard',
 'can i pay by    ***   mastercard',
 'i would like to pay with my    ***   mastercard',
 'i   ***    ***  pay with mastercard',
 '***   ***   ***     ***  

In [ ]:
test_prompts_in_test_data = []
for key in test_prompts:
    test_prompts_in_test_data.append(prompt_test_map[key])

test_prompts_in_test_data 

In [ ]:
counter = 0
credit_test_prompts = []
for key in test_prompts:
    #counter += len((prompt_test_map[key]))
    for chid_reponse in prompt_test_map[key]:
        
        if chid_reponse['transcript'] not in grammar[key]:
            credit_test_prompts.append(chid_reponse['transcript'])

counter

In [ ]:
credit_test_prompts

In [ ]:
def generalise_aux_verb(nlp_sent):
    tags = nlp_sent[2]
    words = spacy_words_to_string_array(nlp_sent[0])
    sentence = " ".join(words)

    prio_one = [['PRP', 'MD', 'VB', 'TO']]
    prio_two = [['PRP', 'VBP', 'TO']]

    match = False
    extracted_words = extract_by_pattern(prio_one, tags, words)
    if len(extracted_words) < 1:
        extracted_words = extract_by_pattern(prio_two, tags, words)
        #if len(extracted_words) < 1:
        #    extracted_words = extract_by_pattern(prio_three_question_patterns, tags, words)
    return extracted_words 

def generalise_verb_prep(nlp_sent):
    tags = nlp_sent[2]
    words = spacy_words_to_string_array(nlp_sent[0])
    sentence = " ".join(words)
    #words = " ".join(nlp_sent[0])
    prio_one = [['VBP', 'IN'], ['VB', 'IN']]
    #prio_one_patterns = [['VBP', 'VBG', 'IN'], ['MD', 'VB', 'CD'], ['VBP', 'RB']]
    # Only when no prio_one_pattern fits
    #prio_sec_patterns = [['VBP']]

    #prio_one_patterns = [['NN', 'NNS'], ['NN', 'NN']]
    
    # Only when no prio_one_pattern fits
    #prio_sec_patterns = [['NN'], ['NNS']]
    match = False
    extracted_words = extract_by_pattern(prio_one, tags, words)
    """if len(extracted_words) < 1:
        extracted_words = extract_by_pattern(prio_two, tags, words)
        #if len(extracted_words) < 1:
        #    extracted_words = extract_by_pattern(prio_three_question_patterns, tags, words)
    """
    return extracted_words 

def generalise_verb_det(nlp_sent):
    tags = nlp_sent[2]
    words = spacy_words_to_string_array(nlp_sent[0])
    sentence = " ".join(words)
    #words = " ".join(nlp_sent[0])
    prio_one = [['VBP', 'DT'], ['VB', 'DT']]
    #prio_one_patterns = [['VBP', 'VBG', 'IN'], ['MD', 'VB', 'CD'], ['VBP', 'RB']]
    # Only when no prio_one_pattern fits
    #prio_sec_patterns = [['VBP']]

    #prio_one_patterns = [['NN', 'NNS'], ['NN', 'NN']]
    
    # Only when no prio_one_pattern fits
    #prio_sec_patterns = [['NN'], ['NNS']]
    match = False
    extracted_words = extract_by_pattern(prio_one, tags, words)
    """if len(extracted_words) < 1:
        extracted_words = extract_by_pattern(prio_two, tags, words)
        #if len(extracted_words) < 1:
        #    extracted_words = extract_by_pattern(prio_three_question_patterns, tags, words)
    """
    return extracted_words 

In [ ]:
found_aux_verb = []
found_verb_prep = []
found_verb_det = []
for key in grammar:
    for item in grammar[key]:
        nlp_s = nlp_sentence(item)
        aux_verb = generalise_aux_verb(nlp_s)
        if len(aux_verb) > 0:
            found_aux_verb.extend(aux_verb)
            
        verb_prep = generalise_verb_prep(nlp_s)
        if len(verb_prep) > 0:
            found_verb_prep.extend(verb_prep)
            
        verb_det = generalise_verb_det(nlp_s)
        if len(verb_det) > 0:
            found_verb_det.extend(verb_det)
        
print(len(set(found_aux_verb)))
print(len(set(found_verb_prep)))
print(len(set(found_verb_det)))

In [ ]:
print(len(set(found_aux_verb)))
print(set(found_aux_verb))
print(len(set(found_verb_prep)))
print(set(found_verb_prep))
print(len(set(found_verb_det)))
print(set(found_verb_det))

In [ ]:
print(list(set(found_verb_det)))

In [ ]:
sentence = "i 'd like to pay with post card"


def aux_key(sentence):
    for auxkey in set(found_aux_verb):
        if auxkey in sentence:
            return auxkey


def verb_prep_key(sentence):
    for verbprepkey in set(found_verb_prep):
        if verbprepkey in sentence:
            return verbprepkey
        
def inside_nouns(sentence):
    for nounkey in sorted(set(nouns), key=len, reverse=True):
        if nounkey in sentence:
            return nounkey
        
def accept_credit_card(sentence):
    aux = aux_key(sentence)
    if aux:
        sentence = sentence.replace(aux, "")
        verb_prep = verb_prep_key(sentence)
        if verb_prep:
            sentence = sentence.replace(verb_prep, "")
            sentence= " ".join(sentence.split(None))
            nounk = inside_nouns(sentence)
            if nounk:
                sentence = sentence.replace(nounk, "")
                if len(sentence) == 0:
                    return True
    return False


In [ ]:
accept_credit_card("i'd like to pay with post card")

In [ ]:
("i'd like to pay with post card".replace("'", " '"))

In [ ]:
counter = 0
for pro in credit_test_prompts:
    sentence = " ".join(pro.split(None))
    if "'" in sentence:
        sentence = sentence.replace("'", " '")
    if accept_credit_card(sentence) == False:
        counter +=1
        print(pro)
print(counter)

In [ ]:

generalise_aux_verb(nlp_sentence("i would like to pay with master card"))

In [ ]:
nlp_sentence("i would need to go tomorrow evening")

In [ ]:
generalise_verb_prep(nlp_sentence("i pay by visa"))

In [ ]:
generalise_verb_det(nlp_sentence("do you have a ticket to westminster"))

In [ ]:
test_prompts

# Generalize it

In [564]:
test_prompts = ["Sag: Ich möchte mit Dollars bezahlen",
"Sag: Ich möchte mit Euros bezahlen",
"Sag: Ich möchte mit Kreditkarte bezahlen",
"Sag: Ich möchte mit Mastercard bezahlen",
"Sag: Ich möchte mit Postkarte bezahlen",
"Sag: Ich möchte mit Visa bezahlen",
"Sag: Ich möchte mit Pfund bezahlen",
"Sag: Ich möchte mit Schweizer Franken bezahlen"]

def get_test_prompts_transcripts_by_cluster(cluster_prompts, prompt_test_data_map):
    # Hole alle Daten mit den prompts
    transcripts_arr = []
    for key in cluster_prompts:
        for response in prompt_test_data_map[key]:
            if response['transcript'] not in grammar[key]:
                transcripts_arr.append(response['transcript'])
    #for key in arr:
    #    if arr[]
    #    transcripts_arr.append()
    return transcripts_arr
    
len(get_test_prompts_transcripts_by_cluster(test_prompts, prompt_test_map))

71

In [489]:
def generate_nouns_etc(cluster_prompts):
    nouns = []
    question_verbs = []
    verbs = []
    for key in cluster_prompts:
        print(key)
        for item in grammar[key]:
            nlp_se = nlp_sentence(item)

            nouns.extend(extract_key_nouns_credit_card(nlp_se))
            question_verbs.extend(extract_key_verbs_question_credit_card(nlp_se))
            verbs.extend(extract_key_verbs_credit_card(nlp_se))
    print(list(set(nouns)))
    print(list(set(question_verbs)))
    print(list(set(verbs)))
generate_nouns_etc(test_prompts)

Sag: Ich möchte mit Dollars bezahlen
Sag: Ich möchte mit Euros bezahlen
Sag: Ich möchte mit Kreditkarte bezahlen
Sag: Ich möchte mit Mastercard bezahlen
Sag: Ich möchte mit Postkarte bezahlen
Sag: Ich möchte mit Visa bezahlen
Sag: Ich möchte mit Pfund bezahlen
Sag: Ich möchte mit Schweizer Franken bezahlen
['card', 'credit card', 'pounds', 'mastercard', 'visa', 'francs', 'dollars', 'cards', 'master card', 'euros', 'credit cards', 'post card']
['is it possible to pay', 'do you accept', 'can i pay with', 'can i pay', 'can i pay by']
['would like to pay with', 'would like to pay by']


In [486]:
sentence = "i 'd like to pay with post card"

def aux_key(sentence):
    for auxkey in set(found_aux_verb):
        if auxkey in sentence:
            return auxkey


def verb_prep_key(sentence):
    for verbprepkey in set(found_verb_prep):
        if verbprepkey in sentence:
            return verbprepkey
        
def inside_nouns(sentence):
    for nounkey in sorted(set(nouns), key=len, reverse=True):
        if nounkey in sentence:
            return nounkey
        
def accept_credit_card(sentence):
    aux = aux_key(sentence)
    if aux:
        sentence = sentence.replace(aux, "")
        verb_prep = verb_prep_key(sentence)
        if verb_prep:
            sentence = sentence.replace(verb_prep, "")
            sentence= " ".join(sentence.split(None))
            nounk = inside_nouns(sentence)
            if nounk:
                sentence = sentence.replace(nounk, "")
                if len(sentence) == 0:
                    return True
    return False

In [488]:
for sent in get_test_prompts_transcripts_by_cluster(["Sag: Ich möchte mit Schweizer Franken bezahlen"], prompt_test_map):
    print(str(accept_credit_card(sent)) + "\t" + sent)

False	i would like to pay with master with master card
False	i would like to pay by have from france
False	can i  ***   am from france
False	***   can i   i buy three nights
False	i like i   with swiss francs


In [487]:
list(get_test_prompts_transcripts_by_cluster(["Sag: Ich möchte mit Schweizer Franken bezahlen"], prompt_test_map))

['i would like to pay with master with master card',
 'i would like to pay by have from france',
 'can i  ***   am from france',
 '***   can i   i buy three nights',
 'i like i   with swiss francs']

## Cluster Restarant

In [491]:
test_prompts= ["Frag: Ich möchte die Rechnung", "Frag: Ich möchte die Dessertkarte"]
list(get_test_prompts_transcripts_by_cluster(test_prompts, prompt_test_map))

['i would like the bill',
 'i would like to pay',
 'i would like the check',
 'can i have the bill',
 'can i have the bill',
 'can i have the bill',
 ' no i have you the gym',
 'yes can i have the dessert card',
 'can i have the dessert card',
 'i would like the dessert card',
 'i would like the is there card',
 'i would like a    ticket to ',
 ' i dessert menu',
 'can i have a   dessert card',
 'i would like to the dessert card',
 'can i have the dessert menu',
 'can i have the dessert menu    *** ',
 'can i have the dessert menu',
 'can i have the dessert my room',
 'can i   have the dessert menu please can i have the dessert menu please',
 'can i have the dessert menu',
 'can i have the dessert menu',
 'can i have the dessert menu pool ',
 'can i have the    ***    swimming pool ']

In [558]:
def generalise_aux_verb(nlp_sent):
    tags = nlp_sent[2]
    words = spacy_words_to_string_array(nlp_sent[0])
    sentence = " ".join(words)
    #words = " ".join(nlp_sent[0])
    prio_one = [['PRP', 'MD', 'VB', 'TO'], ['PRP', 'MD', 'VB', 'DT']]
    prio_two = [['PRP', 'VBP', 'TO']]

    #prio_one_patterns = [['VBP', 'VBG', 'IN'], ['MD', 'VB', 'CD'], ['VBP', 'RB']]
    # Only when no prio_one_pattern fits
    #prio_sec_patterns = [['VBP']]

    #prio_one_patterns = [['NN', 'NNS'], ['NN', 'NN']]
    
    # Only when no prio_one_pattern fits
    #prio_sec_patterns = [['NN'], ['NNS']]
    match = False
    extracted_words = extract_by_pattern(prio_one, tags, words)
    if len(extracted_words) < 1:
        extracted_words = extract_by_pattern(prio_two, tags, words)
        #if len(extracted_words) < 1:
        #    extracted_words = extract_by_pattern(prio_three_question_patterns, tags, words)
    return extracted_words

def extract_by_pattern(patterns, tags, words):
    extracted_words = []
    tags_string = " ".join(tags)
    for pattern in patterns:
        
        pattern_string = " ".join(pattern)
        if is_slice_in_list(pattern, tags):
            pattern_start_index = tags.index(pattern[0])
            if len(pattern) > 1:
                pattern_end_index = tags.index(pattern[-1], pattern_start_index+1)
                if pattern_end_index - pattern_start_index +1 != len(pattern):
                    pattern_end_index = tags.index(pattern[-1], pattern_end_index+1)
                    
                if pattern_end_index - pattern_start_index + 1 == len(pattern):
                    extracted_part = ""
                    for i in range(pattern_start_index, pattern_end_index+1):
                        extracted_part += words[i] + " "
                        

                    extracted_words.append(extracted_part[:-1])
            else:
                extracted_words.append(words[pattern_start_index])
    return extracted_words

def generalise_aux_verb_questions(nlp_sent):
    tags = nlp_sent[2]
    words = spacy_words_to_string_array(nlp_sent[0])
    sentence = " ".join(words)
    #words = " ".join(nlp_sent[0])
    prio_one = [['MD', 'PRP', 'VB', 'DT'], ['MD', 'PRP', 'VB', 'PRP']]
    prio_two = [['MD', 'PRP', 'VB']]
    #prio_two = [['PRP', 'VBP', 'TO']]

    #prio_one_patterns = [['VBP', 'VBG', 'IN'], ['MD', 'VB', 'CD'], ['VBP', 'RB']]
    # Only when no prio_one_pattern fits
    #prio_sec_patterns = [['VBP']]

    #prio_one_patterns = [['NN', 'NNS'], ['NN', 'NN']]
    
    # Only when no prio_one_pattern fits
    #prio_sec_patterns = [['NN'], ['NNS']]
    match = False
    extracted_words = extract_by_pattern(prio_one, tags, words)
    if len(extracted_words) < 1:
        extracted_words = extract_by_pattern(prio_two, tags, words)
        #if len(extracted_words) < 1:
        #    extracted_words = extract_by_pattern(prio_three_question_patterns, tags, words)
    return extracted_words 

generalise_aux_verb_questions(nlp_sentence("would you bring me bill"))

['would you bring me']

In [555]:
#print(nlp_sentence("would you bring me bill"))


In [556]:
print(nlp_sentence("would you bring me bill"))
generalise_aux_verb_questions(nlp_sentence("would you bring me bill"))

([would, you, bring, me, bill], ['would', 'you', 'bring', 'me', 'bill'], ['MD', 'PRP', 'VB', 'PRP', 'NN'], ['MD', 'PRP', 'VB', 'PRP', 'NN'])
['MD', 'PRP', 'VB', 'PRP']


['would you bring me']

In [559]:
rest_verb_questions = []

for key in grammar:
    for item in grammar[key]:
        nlp_s = nlp_sentence(item)
        extracted_verb_question = generalise_aux_verb_questions(nlp_s)
        if len(extracted_verb_question) > 0:
            rest_verb_questions.extend(extracted_verb_question)   

print(len(set(rest_verb_questions)))

33


In [560]:
list(set(rest_verb_questions))

['can you tell me',
 'can i have a',
 'could i have s',
 'can you give me',
 'can i have the',
 'can i buy a',
 'can i pay',
 'could i have some',
 'could you offer me',
 'can you bring me',
 'could i have an',
 'can i have s',
 'can i find a',
 'could i have a',
 'can i buy',
 'can i buy some',
 'can i have an',
 'could you give me',
 'can you offer me',
 'could i have',
 'would you bring me',
 'would you give me',
 'can you show me',
 'could i buy some',
 'can i have',
 'can i have some',
 'could i buy',
 'could you tell me',
 'can i find the',
 'could i buy a',
 'could you bring me',
 'could i have the',
 'can i see the']

In [515]:
test_prompts = ["Frag: Ich möchte die Rechnung", "Frag: Ich möchte die Dessertkarte"]
rest_nouns= []
rest_verb_questions = []
for key in test_prompts:
    for response in grammar[key]:
        nlp_s = nlp_sentence(response)
        extracted_nouns = extract_key_nouns_credit_card(nlp_s)
        if len(extracted_nouns) > 0:
            rest_nouns.extend(extracted_nouns)
         

print(list(set(rest_verb_questions)))
print(list(set(rest_nouns)))

['could you give', 'could i have', 'can you offer', 'can you bring', 'can you give', 'would you bring', 'could you offer', 'can i have', 'could you bring', 'can i see the']
['dessert menu', 'bill', 'check']


In [498]:
found_aux_verb = []
for key in grammar:
    for item in grammar[key]:
        nlp_s = nlp_sentence(item)
        aux_verb = generalise_aux_verb(nlp_s)
        if len(aux_verb) > 0:
            found_aux_verb.extend(aux_verb)
        
        
print(len(set(found_aux_verb)))
#print(len(set(found_verb_prep)))
#print(len(set(found_verb_det)))

32


In [507]:
if "i would like the" in "i would like the bill":
    print(True)

True


In [500]:
print(list(set(found_aux_verb)))

['i will buy these', 'i would like an', 'i will have these', 'i want to', "i 'd need a", 'i would like a', 'i will have the', "i 'd like to", 'i would like some', 'i will have an', 'i should like a', 'i will have a', "i 'd like a", 'i should like some', 'i need to', 'i have to', "i 'd want a", "i 'd like some", 'i will take some', 'i should like an', "i 'd like an", 'i should like to', 'i would need a', 'i would want a', 'i would like the', "i 'd like the", 'i will have some', 'i will take the', 'i will take an', 'i will take a', 'i would like to', 'i would need to']


## Cluster No Reservation

In [562]:
test_prompts = ["Sag: Ich habe keine Reservation"]
list(get_test_prompts_transcripts_by_cluster(test_prompts, prompt_test_map))

['no i pay is there a reservation',
 '***  ***  do not have a reservation',
 '  a tomorrow at nine   ',
 'i have no reservation',
 'i have a   room for have a      room   ',
 "i haven't got a reservation",
 'i have not a reservation',
 "***  i'm mamma have a reservation",
 '***  ***   a have a reservation',
 'where no i have not have i   have no',
 "no i don't have a reservation any",
 'no i  ***  ***  have a      ***    ',
 "no i don't no i do not have a reservation",
 'i have no reservation',
 "no i don't no i do not have a reservation",
 'no i     am good',
 'no i have no reservation',
 'no i    ***     am a reservation',
 'i   ***   have a reservation',
 "i haven't a reservation",
 "i don't have a reservation is bern",
 "i don't where is bern   ",
 'i have a   thursday and    ',
 'i have not a reservation',
 "the i don't have a reservation i pay by is my name is    ",
 'i have not a reservation',
 'i want a reservation',
 'i do not  ***  reservation',
 'i have not a reservation',
 

In [563]:
nlp_sentence("i have no reservation")

([i, have, no, reservation],
 ['i', 'have', 'no', 'reservation'],
 ['PRP', 'VBP', 'DT', 'NN'],
 ['PRP', 'VBP', 'DT', 'NN'])